In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [ ]:
data=pd.read_csv("Churn_Modelling.csv")
data.head() 

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
data=data.drop(['RowNumber', 'CustomerId', 'Surname' ], axis=1)


In [ ]:
label_encoder_gender = LabelEncoder()

data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])


In [ ]:
one_hot_encoder_geo = OneHotEncoder(handle_unknown="ignore")

geo_encoded = one_hot_encoder_geo.fit_transform(data[["Geography"]]).toarray()

geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df



,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [ ]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)


In [ ]:

X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']


## Split the data into two parts training and testing sets
X_train, X_test, y_train, y_test=train_test_split(X, y, train_size=0.2, random_state=42)


## Scale these features
scaler= StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [ ]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


#### ANN Regression Problem Statement

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential([
    Dense(64, activation="relu", input_shape=[X_train.shape[1], ]),
    Dense(32, activation="relu"),
    Dense(1) ## output layer for regression

])

model.compile(optimizer="adam", loss="mean_absolute_error", metrics=['mae'])
model.summary()


c:\Users\karti\OneDrive\Desktop\projects\annclassification\annclassification\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Set up TensorBoard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True  )
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_data=(X_test, y_test)
    ,callbacks=[early_stopping_callback, tensorboard_callback]
    )




Epoch 1/200
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 102456.9688 - mae: 102456.9688 - val_loss: 99453.8516 - val_mae: 99453.8516
Epoch 2/200
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 103429.6016 - mae: 103429.6016 - val_loss: 99436.2969 - val_mae: 99436.2969
Epoch 3/200
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 103900.8594 - mae: 103900.8594 - val_loss: 99388.0625 - val_mae: 99388.0625
Epoch 4/200
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 102442.7891 - mae: 102442.7891 - val_loss: 99289.5547 - val_mae: 99289.5547
Epoch 5/200
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 103387.1250 - mae: 103387.1250 - val_loss: 99121.2812 - val_mae: 99121.2812
Epoch 6/200
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 102701.2109 - mae: 102701.2109 - val_loss: 98864.9375 - val_mae: 98864.9375
Epoch 7/200
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 102296.7031 - mae: 102296.7031 - val_loss: 98499.0156 - val_mae: 98499.0156
Epoch 8/200
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/s

In [ ]:

%reload_ext tensorboard


In [ ]:

%tensorboard --logdir regressionlogs/fit --port=6007

Launching TensorBoard...

In [ ]:
## Evaluate model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')

250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 50459.3906 - mae: 50459.3906
Test MAE : 50257.390625


In [ ]:
model.save('regression_model.h5')